<a href="https://colab.research.google.com/github/redstoneyard/covid_sars/blob/main/covid_sars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import numpy as np
import string
import pandas as pd
import tensorflow as tf
from pandas import DataFrame

up = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

def onehotify(A):
  x=[0.0]*26
  x[up.index(A)] = 1.0
  return x

def onehottensor(seq, leng):
  oht = [[0.0]*26]*leng
  for i in range(0, len(seq)):
    oht[i] = (onehotify(seq[i]))
  oht = np.array(oht)
  return oht  #output is np array

def let_to_int(seq):
  int_seq = []
  for letter in seq:
    int_seq.append(up.index(letter))
  int_seq = np.array(int_seq)
  return int_seq

def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(1, input_shape=(None, 26))  #LSTM, SimpleRNN                           
  ])

  model.compile(optimizer = 'adam',
                loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  
  return model


df = pd.read_csv('/content/drive/My Drive/covid-sars/input_bcell.csv')
test_df = pd.read_csv('/content/drive/My Drive/covid-sars/input_sars.csv')

end = df.pop('end_position')
start = df.pop('start_position')
maxlength1 = (end - start).max() + 1

end = test_df.pop('end_position')
start = test_df.pop('start_position')
maxlength2 = (end - start).max() + 1

maxlength = max(maxlength1, maxlength2)


target = df.pop('target')  #this belongs to class 'pandas.core.series.Series'
target = target.tolist()
for i in range(0,len(target)):
  target[i] = np.array([target[i]])
target = np.array(target)

test_target = test_df.pop('target')
test_target = test_target.tolist()
for i in range(0,len(test_target)):
  test_target[i] = np.array(test_target[i])
test_target = np.array(test_target)


df = df['peptide_seq']     #this belongs to class 'pandas.core.series.Series'
df = df.tolist()
for i in range(0,len(df)):  
  df[i] = onehottensor(df[i], maxlength)
df = np.array(df)

test_df= test_df['peptide_seq']
test_df = test_df.tolist()
for i in range(0, len(test_df)):
  test_df[i] = onehottensor(test_df[i], maxlength)
test_df = np.array(test_df)


print(target.shape)
print(df.shape)



model = get_compiled_model()
print(model.summary())

model.fit(df, target, batch_size = 1, epochs = 1)


model.evaluate(test_df, test_target)








(14387, 1)
(14387, 393, 26)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 1)                 28        
Total params: 28
Trainable params: 28
Non-trainable params: 0
_________________________________________________________________
None
17/17 [==============================] - 0s 16ms/step - loss: 0.5830 - accuracy: 0.7308


[0.5830320119857788, 0.7307692170143127]

df[['peptide_seq']] is a dataframe object

df['peptide_seq'] is a series object.

DataFrame only supports 2-d

np.array only supports rectuangular shape


parent_protein_id       object
protein_seq             object
start_position           int64
end_position             int64
peptide_seq             object
chou_fasman            float64
emini                  float64
kolaskar_tongaonkar    float64
parker                 float64
isoelectric_point      float64
aromaticity            float64
hydrophobicity         float64
stability              float64
target                   int64
dtype: object